<a href="https://colab.research.google.com/github/phsoaressantos/infrahadoop/blob/main/Projeto_Infra_Hadoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://blog.infnet.com.br/wp-content/uploads/sites/50/2021/12/Substituir-Pexels-by-Anna-2048x1365.jpg" width="300" alt="cognitiveclass.ai logo" />
</center>

#**Pós Graduação em Engenharia de Dados: Big Data**

#**Projeto conclusão curso: Infra Hadoop**

Autor: Paulo Henrique Soares dos Santos



#**Dados**

## Visão geral dos dados

- Tabela Exportação:

CO_ANO = Year

CO_MES = Month

CO_NCM = NCM (Code description of the product - Mercosul Common Nomeclature)

CO_UNID = Statistics Unity Code

CO_PAIS = Code of the Destination Country

SG_UF_NCM = UF Code of the product origin

CO_VIA = Via of Transportation Code

CO_URF= URF Code of Embark and Disembark

QT_ESTAT = Statistic Unity

KG_LIQUIDO = Liquid Kg

VL_FOB = Value of Transaction (FOB Dolar US$)


- Tabela País:


CO_PAIS = Country Code

CO_PAIS_ISON3 = ISO 3 Country Code (Numerical)

CO_PAIS_ISOA3 = ISO 3 Country Code (Letters)

NO_PAIS = Country Name in Portugue (BR)

NO_PAIS_ING = Country Name in English

NO_PAIS_ESP = Country Name in Spanish

##Contexto

Este conjunto de dados é fornecido pelo Ministério da Indústria, Comércio Externo e Serviços do Brasil.
Fornecem um site onde se pode fazer algumas perguntas simples, por isso se precisar de algo específico/muito simples, provavelmente pode obter de lá directamente: http://comexstat.mdic.gov.br/

Conteúdo

Estes dados vão de 1997 a Julho/2020 e são actualizados mensalmente no website comex stat.
Os dados são organizados pela NCM (Nomenclatura Comum para produtos de mercosul)


## Fonte Dados

Kaggle :

1 tabela (exportacao): https://www.kaggle.com/datasets/marcelopesse/brazil-exportimport-information?select=EXP_COMPLETA.csv *1

2 tabela (pais): https://www.kaggle.com/datasets/marcelopesse/brazil-exportimport-information?select=PAIS.csv

*1 Editei o CSV original devido ao tempo de upload na GCP, editei ate a linha 1.048.575.



#**Etapas do projeto** 

##Ambiente cloud
  Cluster criado na GCP utilizando DATAPROC - Cluster no computer engine com as seguintes configurações:

  *Nome: Cluster-b335

  *Local: us-central1

  *Tipo de Cluster: Padrão (1 mestre, N workers)

  *Tipo e versão imagem: 2.0debian10

  *Componentes adicionais: Todos, exceto Jupyter notebook

##Carga dados utilizando upload nativo

## Adcionando arquivos no HDFS
hdfs dfs -put PAIS.csv
hdfs dfs -put EXP_COMPLETA_2.csv

#**Hive**
##Acessando Hive
 #entrando no hive beeline, funciona melhor e sem logs

beeline -u jdbc:hive2://localhost:10000/default -n paulo_hssantos@cluster-b335-m -d org.apache.hive.jdbc.HiveDriver

## Criando database trabalho:
  create database trabalho;

  use trabalho;

 #confere a criacao:

  show databases;

##Criando a minha TABELA COM EXPORTACAO 2

CREATE EXTERNAL TABLE exportacao(CO_ANO STRING, CO_MES INT, CO_NCM INT, CO_UNID INT, CO_PAIS STRING, SG_UF_NCM STRING, CO_VIA INT, CO_URF STRING, QT_ESTAT INT, KG_LIQUIDO INT, VL_FOB INT) 
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE;

  #conferindo tabela criada:

  show tables;

  describe exportacao;

##Criando a tabela pais
CREATE EXTERNAL TABLE pais(CO_PAIS STRING, CO_PAIS_ISON3 STRING, CO_PAIS_ISOA3 STRING, NO_PAIS STRING, NO_PAIS_ING STRING, NO_PAIS_ESP STRING) 
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ';'
STORED AS TEXTFILE;

  #conferindo tabela criada:

  show tables;

  describe pais;

# Entrando com os dados nas tabelas pais e exportacao: também!

LOAD DATA INPATH '/user/paulo_hssantos/EXP_COMPLETA_2.csv' overwrite into table exportacao;

LOAD DATA INPATH '/user/paulo_hssantos/PAIS.csv' overwrite into table pais;



# Análise de dados

## Perguntas analiticas e respectivas consultas SQL:

1- Média de exportações por país de destino:

select co_pais, (SUM(VL_FOB)/COUNT(*)) AS CO_PAIS FROM trabalho.exportacao group by co_pais order by CO_PAIS;

2- Valor total de exportações em todo período analisado, agrupados por país de destino:

SELECT exportacao.CO_PAIS, NO_PAIS, SUM(VL_FOB)  from exportacao inner join pais on exportacao.CO_PAIS = pais.CO_PAIS group by NO_PAIS order by SUM(VL_FOB) desc;

3- Soma total de exportação por estado brasileiro;

SELECT SG_UF_NCM, SUM(VL_FOB) FROM trabalho.exportacao group by SG_UF_NCM;


4- Numero de exportações por estado brasileiro em ordem descendente;

select SG_UF_NCM,COUNT(*) AS numero_de_exportacoes from trabalho.exportacao GROUP BY SG_UF_NCM ORDER BY numero_de_exportacoes desc;



#Linguagens e ferramentas utilizadas no projeto:

<p align="left">
<a href="https://cloud.google.com" target="_blank" rel="noreferrer"> <img src="https://www.vectorlogo.zone/logos/google_cloud/google_cloud-icon.svg" alt="gcp" width="40" height="40"/> </a>
<a href="https://hadoop.apache.org/" target="_blank" rel="noreferrer"> <img src="https://www.vectorlogo.zone/logos/apache_hadoop/apache_hadoop-icon.svg" alt="hadoop" width="40" height="40"/>
</a> <a href="https://hive.apache.org/" target="_blank" rel="noreferrer"> <img src="https://www.vectorlogo.zone/logos/apache_hive/apache_hive-icon.svg" alt="hive" width="40" height="40"/>
</a> <a href="https://www.mysql.com/" target="_blank" rel="noreferrer"> <img src="https://raw.githubusercontent.com/devicons/devicon/master/icons/mysql/mysql-original-wordmark.svg" alt="mysql" width="40" height="40"/> </a>  
</p>

# Contato: 

📧 paulosantos@id.uff.br

In [ ]:
!pip install kaggle; #caso seja necessário atualizar a biblioteca no google colab

from google.colab import files
files.upload() #enviar o arquivo kaggle.json

#antes de importar o dataset nós iremos provisionar o local de armazenamento
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#Alterar a permissão para evitar o aviso durante a partida da ferramenta Kaggle
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

#Aceitar a competição e copiar o endereço da API para o download do dataset
#!kaggle competitions download -c house-prices-advanced-regression-techniques
!kaggle datasets download -d marcelopesse/brazil-exportimport-information

#Descompactar o arquivo baixado
!unzip \*.zip  && rm *.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle (1).json
 99% 829M/837M [00:06<00:00, 149MB/s]
100% 837M/837M [00:06<00:00, 145MB/s]
Archive:  brazil-exportimport-information.zip
replace EXP_COMPLETA.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: EXP_COMPLETA.csv        
  inflating: IMP_COMPLETA.csv        
  inflating: NCM.csv                 
  inflating: PAIS.csv                
  inflating: PAIS_BLOCO.csv          
  inflating: UF.csv                  
  inflating: UF_MUN.csv              
  inflating: URF.csv                 
  inflating: VIA.csv                 


In [ ]:
!pip install kaggle; #caso seja necessário atualizar a biblioteca no google colab

from google.colab import files
files.upload() #enviar o arquivo kaggle.json

#antes de importar o dataset nós iremos provisionar o local de armazenamento
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#Alterar a permissão para evitar o aviso durante a partida da ferramenta Kaggle
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

#Aceitar a competição e copiar o endereço da API para o download do dataset
#!kaggle competitions download -c house-prices-advanced-regression-techniques
!kaggle datasets download -d datahackers/state-of-data-2021

#Descompactar o arquivo baixado
!unzip \*.zip  && rm *.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json
state-of-data-2021.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  state-of-data-2021.zip
  inflating: State of Data 2021 - Dataset - Pgina1.csv  


In [ ]:
# importar os pacotes necessários
import pandas as pd

# importar o arquivo
df=pd.read_csv("EXP_COMPLETA.csv",sep=';',encoding='latin-1')

# Precisei adicionar para tratar o arquivo, encoding e separação por ; pd.read_csv('exemplo.csv', sep='|', encoding='latin-1')

# ver 5 as primeiras entradas
df.head()


,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
0,1997,3,41043911,15,149,RS,1,1010500,3987,4150,16725
1,1997,5,63019000,10,97,MG,7,145200,0,1002,8420
2,1997,6,87168000,11,586,RS,7,145300,48,153,915
3,1997,8,39162000,10,249,SP,1,817800,0,138,586
4,1997,2,73130000,10,63,SP,7,1010900,0,2714,1972


In [ ]:
df.describe()

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
count,2.239837e+07,2.239837e+07,2.239837e+07,2.239837e+07,2.239837e+07,2.239837e+07,2.239837e+07,2.239837e+07,2.239837e+07,2.239837e+07
mean,2.009861e+03,6.579248e+00,6.292894e+07,1.071480e+01,3.689264e+02,3.474574e+00,8.091142e+05,6.169763e+05,4.859518e+05,1.611167e+05
std,6.453716e+00,3.397364e+00,2.664048e+07,1.587098e+00,2.615533e+02,2.540408e+00,2.184835e+05,4.874537e+08,2.673264e+07,3.206459e+06
min,1.997000e+03,1.000000e+00,1.011010e+06,1.000000e+01,1.300000e+01,0.000000e+00,9.100000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.005000e+03,4.000000e+00,4.005109e+07,1.000000e+01,1.580000e+02,1.000000e+00,8.176000e+05,1.000000e+00,7.000000e+00,2.690000e+02
50%,2.010000e+03,7.000000e+00,7.318160e+07,1.000000e+01,2.750000e+02,4.000000e+00,8.178000e+05,2.300000e+01,1.140000e+02,2.319000e+03
75%,2.015000e+03,1.000000e+01,8.483401e+07,1.100000e+01,5.860000e+02,4.000000e+00,9.107000e+05,5.400000e+02,3.256000e+03,2.072900e+04
max,2.020000e+03,1.200000e+01,9.999710e+07,2.200000e+01,9.990000e+02,1.500000e+01,9.999999e+06,9.908988e+11,1.546575e+10,1.936700e+09


In [ ]:
NCM = df.CO_NCM
NCM

0           41043911
1           63019000
2           87168000
3           39162000
4           73130000
              ...   
22398365    40112090
22398366    40103100
22398367    84219999
22398368    21032090
22398369    90173020
Name: CO_NCM, Length: 22398370, dtype: int64

In [ ]:
df.groupby("CO_NCM").agg('sum')

,CO_ANO,CO_MES,CO_UNID,CO_PAIS,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
CO_NCM,,,,,,,,,
1011010,1713618,5816,9394,362301,4676,750703674,3817,1738731,23858316
1011090,18049,61,99,5718,53,7761856,80,28347,117203
1011100,543813,1814,2992,92129,1405,230097378,1204,541795,5986595
1011900,189932,542,1045,29307,437,79484509,665,223197,429115
1012000,4001,11,22,973,8,1535300,7,686,3953
...,...,...,...,...,...,...,...,...,...
99980202,19722074,62879,98200,4611774,69868,6793206246,33681724877,431869712,1257128704
99997101,3989052,13043,19880,628008,17809,1323133431,368929,62929,7739491
99997102,48579958,155957,242210,8704807,217686,16816019017,7110270,35305,142693969
